In [1]:
import json 
import os
import pandas as pd 
from datetime import datetime, timedelta
from pathlib import Path

In [2]:

def split_data(data_to_split:list, by:int) -> list:
    return [data_to_split[i:i + by] for i in range(0, len(data_to_split), by)]

In [3]:
def get_json_file_content(json_file_path:str, key:str=None) -> object:
    """get json file content
    Args:
        json_file_path (str): json file path
    Returns:
        object: json file content
    """
    if Path(json_file_path).exists():
        with open(json_file_path, 'r') as openfile:
            file_content = json.load(openfile)
            if file_content and key:
                try:
                    return file_content.get(key)
                except KeyError as e:
                    print('error',f"{e}")
            return file_content
    print('error', 'file does not found')

In [4]:
def combine_file_content(file_path:str, file_type:str) -> list:
    match file_type:
        case 'json':
            files = os.listdir(file_path)
            files = list(filter(lambda f: f.endswith('.json'), files))
            print(files)
            files = [f"{file_path + x}" for x in files]
            file_contents = []
            for file in files:
                print(file)
                file_content = get_json_file_content(f"{file}")
                file_contents += file_content
            return file_contents
        case 'csv':
            pass

In [5]:
def get_saturdays(start_date, end_date):
    saturdays = pd.bdate_range(start=start_date, end=end_date, freq='C', weekmask='Sat').strftime('%Y-%m-%d').tolist()
    return saturdays

In [6]:
def normalize_urls(urls, dates:list) -> list:
    updated_urls = []
    for date in dates:
        start_date = date 
        end_date = (datetime.strptime(date, '%Y-%m-%d') + timedelta(days=6)).strftime('%Y-%m-%d')
        for url in urls:
            if '.php' in url:
                updated_url = f"{url}&date_debut={start_date}&date_fin={end_date}"
            else:
                residence_cle = url.split('_')[-1].split('.')[0]
                updated_url = f"{url}?date_debut={start_date}&date_fin={end_date}&residence_cle={residence_cle}"
            updated_urls.append(updated_url)
    return updated_urls

In [7]:
start_date = "03/03/2025"
end_date = "30/10/2025"
dest_folder_path = "/home/keller/Documents/Jobdev/G2A/Pricing/dests/maeva/setups/new_urls/diff/"
output_path = "/home/keller/Documents/Jobdev/G2A/demo/"
name = "dest"
devided_by = 35

In [8]:
dest_data = combine_file_content(dest_folder_path, 'json')

['all_diff_urls.json']
/home/keller/Documents/Jobdev/G2A/Pricing/dests/maeva/setups/new_urls/diff/all_diff_urls.json


In [9]:
len(dest_data) * len(get_saturdays(start_date, end_date)) // 35

53968

In [10]:
len(dest_data) // 18

3086

In [11]:
cleaned_urls = []

for url in dest_data:
    if '.php' in url:
        cleaned_urls.append(url.split('&')[0])
    else:
        cleaned_urls.append(url.split('?')[0])

In [12]:
print(len(cleaned_urls))
cleaned_urls = list(set(cleaned_urls))
print(len(cleaned_urls))

55556
14777


In [13]:
saturdays = get_saturdays(start_date, end_date)
print(saturdays)

['2025-03-08', '2025-03-15', '2025-03-22', '2025-03-29', '2025-04-05', '2025-04-12', '2025-04-19', '2025-04-26', '2025-05-03', '2025-05-10', '2025-05-17', '2025-05-24', '2025-05-31', '2025-06-07', '2025-06-14', '2025-06-21', '2025-06-28', '2025-07-05', '2025-07-12', '2025-07-19', '2025-07-26', '2025-08-02', '2025-08-09', '2025-08-16', '2025-08-23', '2025-08-30', '2025-09-06', '2025-09-13', '2025-09-20', '2025-09-27', '2025-10-04', '2025-10-11', '2025-10-18', '2025-10-25']


In [14]:
number_data_per_file = len(cleaned_urls) // devided_by

In [15]:
splited_datas = split_data(cleaned_urls, number_data_per_file)
splited_datas

[['https://www.maeva.com/fr-fr/pages/fiche.php?id=807744',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=1206651',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=833154',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=772634',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=797494',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=328274',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=807244',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=187626',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=779334',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=1505371',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=1402831',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=1066364',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=1706831',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=376174',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=415954',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=187561',
  'https://www.maeva.com/fr-fr/pages/fiche.php?id=1

In [16]:
new_splited_datas = [normalize_urls(data, saturdays) for data in splited_datas]

In [17]:
for i in range(len(new_splited_datas)):
    with open(f"{output_path}{name}_{i+1}.json", 'w') as outfile:
        json.dump(new_splited_datas[i], outfile)

last changed